While the Data Submission Portal API is primarily intended to support submissions, the information that can be returned is also useful for reporting an monitoring a collection of submissions.  In the notebooks, we'll demonstrate some of the queries that can provide information useful for seeing the current state of your submisison universe

In [38]:
import requests
import pandas as pd
import os
from IPython.display import display, Markdown, Latex

In [3]:
prod = 'https://hub.datacommons.cancer.gov/api/graphql'
#Note that use of Dev2 is for example purposes only.  Unless you are an approved tester, you should be using the production URL
dev2 = 'https://hub-dev2.datacommons.cancer.gov/api/graphql'

In [4]:
def apiQuery(url, query, variables):
    token = os.environ['DEV2API']
    headers = {"Authorization": f"Bearer {token}"}
    try:
        if variables is None:
            result = requests.post(url = url, headers = headers, json={"query": query})
        else:
            result = requests.post(url = url, headers = headers, json = {"query":query, "variables":variables})
        if result.status_code == 200:
            return result.json()
        else:
            print(f"Error: {result.status_code}")
            return result.content
    except requests.exceptions.HTTPError as e:
        return(f"HTTP Error: {e}")

One potentially useful report would be a simple overview of all the studies your organization is working on.  This can be done with the *listApprovedStudiesOfMyOrganization* query.

In [6]:
study_query = """
{
  listApprovedStudiesOfMyOrganization{
    originalOrg
    studyName
    studyAbbreviation
    dbGaPID
    controlledAccess
  }
}
"""

In [7]:
studies_res = apiQuery(dev2,study_query,None)
studies_df = pd.DataFrame(studies_res['data']['listApprovedStudiesOfMyOrganization'])
display(Markdown(studies_df.to_markdown()))

|    | originalOrg                                    | studyName                                                                                       | studyAbbreviation   | dbGaPID   | controlledAccess   |
|---:|:-----------------------------------------------|:------------------------------------------------------------------------------------------------|:--------------------|:----------|:-------------------|
|  0 | Purdue Center for Cancer Research              | Antitumor Activity and Molecular Effects of Vemurafenib in Dogs with BRAF-mutant Bladder Cancer | UBC01               |           | False              |
|  1 | Comparative Molecular Characterization Program | A Multi-Platform Sequencing Analysis of Canine Appendicular Osteosarcoma                        | OSA01               |           | False              |
|  2 | Comparative Molecular Characterization Program | Whole exome sequencing analysis of canine cancer cell lines                                     | TCL01               |           | False              |
|  3 | NCI BBRB                                       | Cancer Moonshot Biobank                                                                         | CMB                 |           | False              |
|  4 | CCDI                                           | T-cell Acute Lymphoblastic Leukemia Single Cell RNA Sequencing and ATAC Sequencing              | TALLsc              | phs003432 | True               |

While this result provides a high-level overview, it doesn't really dig into the details of what's really going on.  To get to a more fine-grained look at the actual submissions, the *listSubmissions* query can delve into much detail. Note that the *listSubmissions* query requires the **status** parameter which can be one of the following:
- All
- New
- In Progress
- Submitted
- Released
- Completed
- Archived
- Canceled
- Rejected
- Withdrawn
- Deleted
Since we're reporting on the overall state of all submissions, we'll use **All** in the query.

In [46]:
listsubmissions_query = """
query ListSubmissions($status: String!){
  listSubmissions(status: $status){
    submissions{
      name
      submitterID
      submitterName
      studyAbbreviation
      studyID
      dbGaPID
      createdAt
      updatedAt
      metadataValidationStatus
      fileValidationStatus
      status
    }
  }
}
"""

In [47]:
submisisons_variable = {"status":"All"}

In [48]:
submission_res = apiQuery(dev2,listsubmissions_query,submisisons_variable)
submission_df = pd.DataFrame(submission_res['data']['listSubmissions']['submissions'])
display(Markdown(submission_df.to_markdown()))

|    | name                           | submitterID                          | submitterName          | studyAbbreviation   | studyID                              | dbGaPID   | createdAt                | updatedAt                | metadataValidationStatus   | fileValidationStatus   | status      |
|---:|:-------------------------------|:-------------------------------------|:-----------------------|:--------------------|:-------------------------------------|:----------|:-------------------------|:-------------------------|:---------------------------|:-----------------------|:------------|
|  0 | Jupyter Demo 4                 | 5fd61de6-4843-4a28-9478-dd070342de47 | todd.pihl              | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-09-03T20:14:35.713Z | 2024-09-04T17:13:24.196Z | Error                      |                        | In Progress |
|  1 | Jupyter Demo 4                 | 5fd61de6-4843-4a28-9478-dd070342de47 | todd.pihl              | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-09-04T17:10:34.607Z | 2024-09-04T17:10:34.607Z |                            |                        | New         |
|  2 | Jupyter Demo 3                 | 5fd61de6-4843-4a28-9478-dd070342de47 | todd.pihl              | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-08-30T12:44:06.585Z | 2024-09-03T17:20:51.915Z | Error                      |                        | In Progress |
|  3 | Jupyter Demo 2                 | 5fd61de6-4843-4a28-9478-dd070342de47 | todd.pihl              | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-08-28T17:09:15.982Z | 2024-08-28T17:09:54.717Z |                            |                        | In Progress |
|  4 | Jupyter Demo 1                 | 5fd61de6-4843-4a28-9478-dd070342de47 | todd.pihl              | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-08-28T16:38:20.788Z | 2024-08-28T16:41:34.800Z |                            |                        | In Progress |
|  5 | Loading Files V2               | 59960f17-6efa-4ecc-89b3-fed832ec5e38 | Alec (Submitter) Mattu | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-08-28T15:55:05.721Z | 2024-08-28T15:56:05.607Z | Passed                     |                        | In Progress |
|  6 | non-file-node                  | fd63018c-2f0a-4627-8508-fb50bcebda1c | Young Yoo              | CMB                 | 4c2b6522-20b8-4841-8c7a-318b325c99b4 |           | 2024-08-23T14:56:36.482Z | 2024-08-28T14:40:40.586Z | New                        |                        | In Progress |
|  7 | Test                           | 59960f17-6efa-4ecc-89b3-fed832ec5e38 | Alec (Submitter) Mattu | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-08-28T13:07:20.620Z | 2024-08-28T13:18:41.934Z | Error                      |                        | Canceled    |
|  8 | Demo create submission Jupyter | 5fd61de6-4843-4a28-9478-dd070342de47 | todd.pihl              | TALLsc              | 49a69fef-71f8-44e6-ad3b-f7a62d91e348 | phs003432 | 2024-08-27T19:24:25.218Z | 2024-08-28T12:40:12.877Z |                            |                        | Canceled    |
|  9 | CRDCDH-1545                    | 59960f17-6efa-4ecc-89b3-fed832ec5e38 | Alec (Submitter) Mattu | UBC01               | b9e9ab79-d90b-4ec1-83b7-f83a5a75f5b5 |           | 2024-08-27T19:40:52.619Z | 2024-08-27T19:41:08.964Z |                            |                        | Canceled    |

One of the metrics that submitters need to pay attention to is the inactivity time on a submission.  Any submission that is inactive for 60 days will start receiving warnings and any submission that is inactive for 180 days will be deleted.  So having a dashboard of inactivity may help with spotting submissions that are in trouble before they get deleted.

Lets create three bins of submissions, those that have been inactive for <60 days, those that have been inactive for >60 days but <150, and then those that are at risk and are >150.

In [60]:
from datetime import datetime, timezone

In [53]:
submission_df['createdAt'] = pd.to_datetime(submission_df['createdAt'])
submission_df['updatedAt'] = pd.to_datetime(submission_df['updatedAt'])

In [69]:
days = []
for index, row in submission_df.iterrows():
    update = row['updatedAt']
    now = datetime.now(timezone.utc)
    diff = (now - update).days
    days.append(diff)
submission_df.insert(8,'inactiveDays',days,True)

In [70]:
submission_df.head()

,name,submitterID,submitterName,studyAbbreviation,studyID,dbGaPID,createdAt,updatedAt,inactiveDays,metadataValidationStatus,fileValidationStatus,status
0,Jupyter Demo 4,5fd61de6-4843-4a28-9478-dd070342de47,todd.pihl,TALLsc,49a69fef-71f8-44e6-ad3b-f7a62d91e348,phs003432,2024-09-03 20:14:35.713000+00:00,2024-09-04 17:13:24.196000+00:00,1,Error,None,In Progress
1,Jupyter Demo 4,5fd61de6-4843-4a28-9478-dd070342de47,todd.pihl,TALLsc,49a69fef-71f8-44e6-ad3b-f7a62d91e348,phs003432,2024-09-04 17:10:34.607000+00:00,2024-09-04 17:10:34.607000+00:00,1,None,None,New
2,Jupyter Demo 3,5fd61de6-4843-4a28-9478-dd070342de47,todd.pihl,TALLsc,49a69fef-71f8-44e6-ad3b-f7a62d91e348,phs003432,2024-08-30 12:44:06.585000+00:00,2024-09-03 17:20:51.915000+00:00,2,Error,None,In Progress
3,Jupyter Demo 2,5fd61de6-4843-4a28-9478-dd070342de47,todd.pihl,TALLsc,49a69fef-71f8-44e6-ad3b-f7a62d91e348,phs003432,2024-08-28 17:09:15.982000+00:00,2024-08-28 17:09:54.717000+00:00,8,None,None,In Progress
4,Jupyter Demo 1,5fd61de6-4843-4a28-9478-dd070342de47,todd.pihl,TALLsc,49a69fef-71f8-44e6-ad3b-f7a62d91e348,phs003432,2024-08-28 16:38:20.788000+00:00,2024-08-28 16:41:34.800000+00:00,8,None,None,In Progress
